<h5 style="direction:rtl;" align="center">بسم الله الرحمن الرحیم</h5>
<br>
<h4 style="direction:rtl;" align="center">دانشگاه صنعتی شریف - بهار ۱۴۰۲</h4>
<h4 style="direction:rtl;" align="center">دانشکده مهندسی کامپیوتر</h4>
<h3 style="direction:rtl;" align="center">فاز اول پروژه درس سامانه‌های یادگیری ماشین</h3>
<hr>
<div><h2 style="direction:rtl;" align="center">قسمت استخراج امبدینگ برای دادگان نمونه</h2><div>
<hr>
<h4 style="direction:rtl;" align="center">اعضای گروه: محمدحسین موثقی‌نیا، مهدی منوچهری، حمید مهتدی جعفری</h4>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from transformers import AutoConfig, AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel
import torch
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import re
pd.options.mode.chained_assignment = None

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<div style="direction:rtl;">در این قسمت دیتا به گوگل درایو اضافه شده است. اگر میخواهید کد اجرا کنید لطفا ادرس را درست کنید.</div>

In [ ]:
# Import data address 
from google.colab import drive
drive.mount('/content/drive')
fresh_sample = pd.read_csv('/content/drive/MyDrive/mlops/mlops_part_1_progect/fresh_sample.csv')
rotten_sample = pd.read_csv('/content/drive/MyDrive/mlops/mlops_part_1_progect/rotten_sample.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Bert Embedding

In [ ]:
print('Encoding review content for fresh sample')
title_encoding = []
model.eval()
for text in tqdm(fresh_sample["review_content"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    title_encoding.append(output.pooler_output[0].detach().numpy())

title_encoding = [t.astype('float64') for t in title_encoding]
fresh_sample.insert(loc=11, column='review_content_bert_embedding', value=title_encoding)


print('Encoding review content for rotten_sample')
title_encoding = []
model.eval()
for text in tqdm(rotten_sample["review_content"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    title_encoding.append(output.pooler_output[0].detach().numpy())

title_encoding = [t.astype('float64') for t in title_encoding]
rotten_sample.insert(loc=11, column='review_content_bert_embedding', value=title_encoding)

Encoding review content for fresh sample


100%|██████████| 100/100 [00:24<00:00,  4.06it/s]


Encoding review content for rotten_sample


100%|██████████| 100/100 [00:16<00:00,  5.99it/s]


In [ ]:
rotten_sample = rotten_sample[["rotten_tomatoes_link","review_type", "review_content", "review_content_bert_embedding"]]
fresh_sample = fresh_sample[["rotten_tomatoes_link","review_type", "review_content", "review_content_bert_embedding"]]

## Tf-idf embedding

In [14]:
all_data = pd.concat([fresh_sample, rotten_sample])
all_data.reset_index(drop=True, inplace=True)
all_data

,rotten_tomatoes_link,review_type,review_content,review_content_bert_embedding
0,m/official_secrets,Fresh,"Like the whistleblower at its center, ""Officia...",[-0.89845914 -0.51225573 -0.85514754 0.692905...
1,m/blue_is_the_warmest_color,Fresh,The film is like a tough exam that everybody a...,[-8.55783403e-01 -4.74810183e-01 -9.47439671e-...
2,m/hardball,Fresh,Hardball is striking in its portrayals of just...,[-0.74963385 -0.61064798 -0.96854866 0.668219...
3,m/pay_2_play_democracys_high_stakes,Fresh,Persuasive and frightening. It's much more ill...,[-0.88150346 -0.34318769 -0.88624769 0.780279...
4,m/aviator,Fresh,Charts three decades in the life of Howard Hug...,[-8.70814800e-01 -5.33839822e-01 -9.48109627e-...
...,...,...,...,...
195,m/i_am_woman,Rotten,'I Am Woman' is a pleasant way to spend a coup...,[-0.79942435 -0.50456363 -0.93217558 0.699359...
196,m/trolls_world_tour,Rotten,"It's a shadow of its predecessor, a regurgitat...",[-0.80640453 -0.33984238 -0.90188742 0.726704...
197,m/love_happens,Rotten,Some may wonder why Jennifer Aniston keeps tak...,[-0.79218954 -0.53846771 -0.96729058 0.689828...
198,m/notebook,Rotten,It's all sweet and gooey and absolutely predic...,[-7.59331882e-01 -2.72824109e-01 -8.80044341e-...


In [21]:
comments = list(all_data['review_content'])
vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2,7), dtype=np.float32)
output = vectorizer.fit_transform(comments).toarray()
all_data['review_content_tf_idf_embedding'] = [t for t in output]

In [31]:
all_data.to_json('sample_embedding.json')